# Задание.

На сайте запущен А/В тест с целью увеличить доход. 
В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.

Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [2]:
from typing import Union # строгая типизация данных
from tqdm import tqdm # прогрес бар для циклов

import pandas as pd
import numpy as np

from scipy import stats # для работы со стат данными и проверки стат гипотез
from statsmodels.stats.meta_analysis import effectsize_smd # для работы со стат данными и проверки стат гипотез
from statsmodels.stats import proportion # для работы со стат данными и проверки стат гипотез
from statsmodels.stats.power import tt_ind_solve_power # для работы со стат данными и проверки стат гипотез
from statsmodels.stats.power import zt_ind_solve_power # для работы со стат данными и проверки стат гипотез

In [3]:
df = pd.read_excel('D:\\Nastya\\GeekBrains\\GeekBrains\\АВ тестирование\\Семинар 8\\gb_sem_8_hm.xlsx')

C:\Users\User\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [14]:
df.head()
# выводим первые 5 строк
# variant - тестовая группа

,USER_ID,VARIANT_NAME,REVENUE
0,2,control,0.0
1,3,control,0.0
2,3,variant,0.0
3,4,variant,0.0
4,5,variant,0.0


In [5]:
df.shape
# смотрим количество строк и столбцов в целом

(10000, 3)

In [6]:
df.USER_ID.unique()
# смотрим количество уникальных (неповторяющихся) значений по столбцу USER_ID

6324

In [7]:
df = df.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})
# группируем по юзер айди и вэриант нэйм и хотим вытащить по столбцу ревенью сумму.

In [8]:
df.shape
# смотрим количество строк и столбцов сформированной выше таблицы

(7865, 3)

In [9]:
df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()
# группируем по юзер айди и хотим вытащить по столбцу вариант нэйм количество по группам тестовой и контрольной (в сумме 6324)

1    4783
2    1541
Name: VARIANT_NAME, dtype: int64

In [10]:
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )

In [11]:
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)
# создаем новый датафрейм в копию БЕЗ дубликатов

In [12]:
df_new.describe()
# смотрим на описание признаков и на их содержание (статистика) нового датафрейма

,USER_ID,REVENUE
count,4783.000000,4783.000000
mean,4994.395777,0.135873
std,2898.618472,3.011392
min,2.000000,0.000000
25%,2476.000000,0.000000
50%,4975.000000,0.000000
75%,7515.000000,0.000000
max,9998.000000,196.010000


In [16]:
gr_A = df[df['VARIANT_NAME'] == 'control'].copy(deep=True) # разделяем по группам А и В, затем вытаскивапем их, потом их копируем в новый датафрейм (дип тру - если у нас будут изменения в оришгинальном датафрейме, то эти изменения никак не повлияют на наши созданные новые массивы)
gr_B = df[df['VARIANT_NAME'] == 'variant'].copy(deep=True)
# treatment - воздействие, это наша тестовая группа

In [17]:
gr_A.describe()

# смотрим по отдельности описательные статистики - контрольная группа (без изменений)

,USER_ID,REVENUE
count,3931.000000,3931.000000
mean,4993.953447,0.163572
std,2894.614378,3.385721
min,2.000000,0.000000
25%,2488.000000,0.000000
50%,4972.000000,0.000000
75%,7574.000000,0.000000
max,10000.000000,196.010000


In [18]:
gr_B.describe()

# смотрим по отдельности описательные статистики - тестовая группа (с изменениями)

,USER_ID,REVENUE
count,3934.000000,3934.000000
mean,4961.772242,0.089342
std,2887.055258,1.484566
min,3.000000,0.000000
25%,2439.750000,0.000000
50%,4956.500000,0.000000
75%,7415.750000,0.000000
max,10000.000000,58.630000


Максимальный доход (revenue) в первой группе сильно больше, чем в группе B. 

Мы заранее видим, что наши метрики различаются, но нам нужно это статистически доказать.

In [24]:
# def continious_result - для непрерывной метрики
def continious_result(gr_A: pd.DataFrame,
                      gr_B: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам (вытащить описательные статистики: размер выборки, среднее контрольной и тестовой групп, станд откл обеих групп)
    size = gr_A.loc[:, column].shape[0]
    
    gr_A_mean = gr_A.loc[:, column].mean()
    gr_B_mean = gr_B.loc[:, column].mean()
    
    gr_A_std = gr_A.loc[:, column].std(ddof=1)
    gr_B_std = gr_B.loc[:, column].std(ddof=1)
    
#  теперь нам нужно посчитать либо при нормальном распределении  Т-критерий, либо при ненормальном - критерий манна уитни
#  ЛИБО пойти другим путем и посчитать через БУТСТРАП
# Преимущества БУТСТРАПА: 
# 1 - с него мы сразу получаем распределение разницы наших средних
# 2 - на его основе можно посчитать через генератор распределение значений p-value
# 3 - если был бы А/А тест, могли бы посчитать значение ошибки 1 рода разниц между нашими группами, будет она или нет.

    # Бутсрап
    booted_diff = [] # создаем лист, куда мы будем записывать БУТСТРАПированную разницу
    for _ in tqdm(range(n_iters)): # запускаем цикл по количеству итераций
        gr_A_sample = gr_A.loc[:, column].sample(n=size, replace=True).values #семплируем контроль (нужно выделить колонку, которую мы семплируем, и передать ему размер выборки, кот мы семплируем)
        gr_B_sample = gr_B.loc[:, column].sample(n=size, replace=True).values # replace=True - чтобы были замены и возвраты, чтобы не было всегда одного и того же результата
        booted_diff.append(np.mean(gr_A_sample - gr_B_sample)) # считаем разницу между контролем и тестом и записываем среднее значение этой разницы в массив booted_diff
# т.о. мы получим распределение бутстрапированный разницы средних наших выборок
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1) # md_ci - средрнее значение (т.е. средний эффект межд значениями в группах), std_ci - стд отклонение этого эффекта 
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5]) # левая и правая границы доверительного интервала
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci))) # считаем значение p-value через генератора норм распр-ия и суммы куммулятивной ф-ии
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=gr_B_mean, sd1=gr_B_std, nobs1=size,
                                    mean2=gr_A_mean, sd2=gr_A_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, # ошибка 1 рода
                           'beta': (1-power), # ошибка 2 рода
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',  # доверит интвервал (лев и прав границы округлим до 3 знаков после запятой)
                           'difference': md_ci,}, # разница между нашими группами
                          index=[column]) 
    return result

# результатом будет табличка

In [23]:
continious_result(gr_A, gr_B, column='REVENUE')

# progress bar (ниже черная строка)

100%|██████████| 10000/10000 [00:06<00:00, 1589.52it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.028393,0.20239,0.757808,"[-0.024, 0.208]",0.073802


In [ ]:
Результаты А/В теста:
    1. Аlpha = 0,20239 = 20% - это p-value, больше чем 5%.
    2. Высокая ошибка 2 рода, т.к. beta = 76%, а должно быть в норме ~ 20%, т.е. 0,02.
    3. Доверительный интервал [-0.024, 0.208] включает в себя 0, это значит, что разница между группами статистически не значима.
    4. Difference (различия/разница) совсем маленькая.
    
На основе полученных данных можно сделать вывод, что гипотеза для увеличения дохода, которую мы выдвинули для А/В теста, не подтвердилась.
Изменения выкатывать нельзя.
Проверить количество юзеров в каждой группе, они должны быть примерно одинаковыми.
Возможно, произошло неправильное семплирование данных.
Рекомендации выбрать другую гипотезу, следуя чек-листу при проведении А/В теста. 
Либо предварительно провести А/А тест.